In [0]:
from delta import DeltaTable

dt = DeltaTable.forPath(spark, "/Volumes/idc/idc_kaggle/ecom_delta_data")
dt.toDF().limit(5).display()

In [0]:
dt.toDF().count()

In [0]:
updated_df = spark.read.csv("/Volumes/idc/idc_kaggle/ecom_data/2019-Nov.csv", inferSchema=True, header=True)
updated_df = updated_df.limit(100000)
updated_df.count()

# MERGE for incremental updates

In [0]:
dt.alias('t').merge(
    updated_df.dropDuplicates(['user_session', 'event_time']).alias('s'),
    "t.user_session = s.user_session AND t.event_time = s.event_time"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

# Time travel

In [0]:
ecom_v0 = spark.read.format('delta').option("versionAsOf", 0).load("/Volumes/idc/idc_kaggle/ecom_delta_data")
ecom_v0.count()

In [0]:
%sql
select count(*) from idc.idc_kaggle.ecom_delta timestamp as of '2026-01-12 17:54:54';

In [0]:
yes = spark.read.format('delta').option("timestampAsOf", "2026-01-13 17:30:18").load("/Volumes/idc/idc_kaggle/ecom_delta_data")
yes.count()

# Optimize

In [0]:
    %sql 
    OPTIMIZE idc.idc_kaggle.ecom ZORDER BY (event_type, user_id);


In [0]:
%sql  DESCRIBE HISTORY idc.idc_kaggle.ecom;

In [0]:
%sql
DESCRIBE DETAIL idc.idc_kaggle.ecom;

In [0]:
%sql
EXPLAIN
SELECT *
FROM idc.idc_kaggle.ecom
WHERE event_type = 'purchase'
  AND user_id = 123456;


In [0]:
%sql
VACUUM idc.idc_kaggle.ecom RETAIN 168 HOURS